# Figuring out how best to impute missing Death data

In [1]:
import pandas as pd

pd.set_option("mode.copy_on_write", True)

In [11]:
# Load in unclean Dataset
mort_pop_unclean = pd.read_parquet(
    "../20_intermediate_files/mort_pop_merge_unclean.parquet", engine="fastparquet"
)

mort_pop_unclean.shape

mort_pop_unclean.sample(5)

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths
25378,01133,3,6,Alabama,Winston County,2012,24193,NaN
33363,31173,2,4,Nebraska,Thurston County,2014,6961,NaN
28850,13065,3,5,Georgia,Clinch County,2013,6758,NaN
25799,13279,3,5,Georgia,Toombs County,2012,27168,NaN
21672,48297,3,7,Texas,Live Oak County,2010,11556,NaN


In [3]:
# Store dataframe with missing deaths
counties_missing_deaths = mort_pop_unclean[mort_pop_unclean["Deaths"].isna()]

counties_missing_deaths.head()

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths
0,01001,3,6,Alabama,Autauga County,2003,46800,NaN
1,01003,3,6,Alabama,Baldwin County,2003,151509,NaN
2,01005,3,6,Alabama,Barbour County,2003,28594,NaN
3,01007,3,6,Alabama,Bibb County,2003,21399,NaN
4,01009,3,6,Alabama,Blount County,2003,53457,NaN


# Using State, Year Death Rate

Method 1: This resulted in some NaN value being imputed to greater than 10 (312 observations were greater than 10 )

Method 2 (Seen below): Used the same method but then forced all death estimates that were over 10 to be 9 in order to be in alignment with the project guidance (i.e. if the county had less than 10 deaths no death data was recorded).

In [4]:
import pandas as pd

# Step 1: Calculate state-year death rate
state_year_totals = (
    mort_pop_unclean.groupby(["ST_NAME", "Year"])
    .agg(total_deaths=("Deaths", "sum"), total_population=("Population", "sum"))
    .reset_index()
)

# Add death rate column
state_year_totals["death_rate"] = (
    state_year_totals["total_deaths"] / state_year_totals["total_population"]
)

# Step 2: Merge death rate back to original dataset
mort_pop_no_missing = mort_pop_unclean.merge(
    state_year_totals[["ST_NAME", "Year", "death_rate"]],
    on=["ST_NAME", "Year"],
    how="left",
)

# Step 3: Impute missing deaths
mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].fillna(
    mort_pop_no_missing["Population"] * mort_pop_no_missing["death_rate"]
)

# Step 4: Round deaths to the nearest integer
mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].round()

# Step 5: Force imputed values greater than 10 to be 9
mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].apply(
    lambda x: 9 if pd.isna(x) is False and x > 10 else x
)

mort_pop_no_missing.head()

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,death_rate
0,01001,3,6,Alabama,Autauga County,2003,46800,0.0,0.0
1,01003,3,6,Alabama,Baldwin County,2003,151509,0.0,0.0
2,01005,3,6,Alabama,Barbour County,2003,28594,0.0,0.0
3,01007,3,6,Alabama,Bibb County,2003,21399,0.0,0.0
4,01009,3,6,Alabama,Blount County,2003,53457,0.0,0.0


In [5]:
# Use the indices of counties_missing_deaths to filter mort_pop_no_missing
imputed_deaths_check = mort_pop_no_missing.loc[counties_missing_deaths.index]

# Display the result
imputed_deaths_check

# Ensure now imputed values are more than 10
imputed_deaths_check[imputed_deaths_check["Deaths"] > 10]

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,death_rate


# Using Death rate by FIPS

Similar results to the using death rate by state, year. I think this method makes less sense though because it is making a death rate from multiple years rather and assigning it back to each year throughout time vs using similar death rates for a given time and place.

NOTE: I did not for values greater than 10 to be 9 as I chose to use the above method instead.


In [6]:
import pandas as pd

# Step 1: Group by FIPS_CODE to calculate total deaths and total population
fips_totals = (
    mort_pop_unclean.groupby("FIPS_CODE")
    .agg(total_deaths=("Deaths", "sum"), total_population=("Population", "sum"))
    .reset_index()
)

# Step 2: Calculate death rate for each FIPS_CODE
fips_totals["fips_death_rate"] = (
    fips_totals["total_deaths"] / fips_totals["total_population"]
)

# Step 3: Merge the calculated death rate back into the original dataset
mort_pop_with_fips_rate = mort_pop_unclean.merge(
    fips_totals[["FIPS_CODE", "fips_death_rate"]], on="FIPS_CODE", how="left"
)

# Step 4: Impute missing deaths using the calculated FIPS_CODE death rate
mort_pop_with_fips_rate["Deaths"] = mort_pop_with_fips_rate["Deaths"].fillna(
    (
        mort_pop_with_fips_rate["Population"]
        * mort_pop_with_fips_rate["fips_death_rate"]
    ).round()
)

mort_pop_with_fips_rate.head()

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,fips_death_rate
0,01001,3,6,Alabama,Autauga County,2003,46800,0.0,0.0
1,01003,3,6,Alabama,Baldwin County,2003,151509,0.0,0.0
2,01005,3,6,Alabama,Barbour County,2003,28594,0.0,0.0
3,01007,3,6,Alabama,Bibb County,2003,21399,0.0,0.0
4,01009,3,6,Alabama,Blount County,2003,53457,0.0,0.0


In [7]:
# Use the indices of counties_missing_deaths to filter mort_pop_no_missing
fips_imputed_deaths_check = mort_pop_with_fips_rate.loc[counties_missing_deaths.index]

# Display the result
fips_imputed_deaths_check

# Ensure now imputed values are more than 10
fips_imputed_deaths_check[fips_imputed_deaths_check["Deaths"] > 10]

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,fips_death_rate
281,10005,3,5,Delaware,Sussex County,2003,168629,14.0,0.000086
289,12015,3,5,Florida,Charlotte County,2003,153235,12.0,0.000081
372,13045,3,5,Georgia,Carroll County,2003,97396,14.0,0.000149
375,13051,3,5,Georgia,Chatham County,2003,237348,18.0,0.000075
421,13139,3,5,Georgia,Hall County,2003,153561,14.0,0.000090
...,...,...,...,...,...,...,...,...,...
32599,21013,3,6,Kentucky,Bell County,2014,27826,13.0,0.000460
33077,28109,3,6,Mississippi,Pearl River County,2014,55121,11.0,0.000194
33597,37157,3,5,North Carolina,Rockingham County,2014,91790,12.0,0.000129
34663,54047,3,5,West Virginia,McDowell County,2014,20385,11.0,0.000516


# Verifying that mort_pop_clean_impute.py works as intended

In [ ]:
import pandas as pd


def impute_and_save_deaths(input_path, output_path):
    # Read the input Parquet file
    mort_pop_unclean = pd.read_parquet(input_path)

    # Identify rows with missing deaths
    counties_missing_deaths = mort_pop_unclean[mort_pop_unclean["Deaths"].isna()]

    # Step 1: Calculate state-year death rate
    state_year_totals = (
        mort_pop_unclean.groupby(["ST_NAME", "Year"])
        .agg(total_deaths=("Deaths", "sum"), total_population=("Population", "sum"))
        .reset_index()
    )

    # Add death rate column
    state_year_totals["death_rate"] = (
        state_year_totals["total_deaths"] / state_year_totals["total_population"]
    )

    # Step 2: Merge death rate back to the original dataset
    mort_pop_no_missing = mort_pop_unclean.merge(
        state_year_totals[["ST_NAME", "Year", "death_rate"]],
        on=["ST_NAME", "Year"],
        how="left",
    )

    # Step 3: Impute missing deaths
    mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].fillna(
        mort_pop_no_missing["Population"] * mort_pop_no_missing["death_rate"]
    )

    # Step 4: Round deaths to the nearest integer
    mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].round()

    # Step 5: Force imputed values greater than 10 to be 9
    mort_pop_no_missing["Deaths"] = mort_pop_no_missing["Deaths"].apply(
        lambda x: 9 if pd.isna(x) is False and x > 10 else x
    )

    # Save the resulting dataset to the output path
    mort_pop_no_missing.to_parquet(output_path, index=False)


if __name__ == "__main__":
    # Define input and output file paths
    input_file = "../20_intermediate_files/mort_pop_merge_unclean.parquet"
    output_file = "../20_intermediate_files/mort_pop_cleaned_imputed.parquet"

    # Run the function
    impute_and_save_deaths(input_file, output_file)

In [12]:
mort_pop_no_missing_test = pd.read_parquet(
    "../20_intermediate_files/mort_pop_cleaned_imputed.parquet", engine="fastparquet"
)

In [15]:
# Use the indices of counties_missing_deaths to filter mort_pop_no_missing
imputed_deaths_check_test = mort_pop_no_missing_test.loc[counties_missing_deaths.index]

# Display the result
display(imputed_deaths_check_test)

# Ensure now imputed values are more than 10
display(imputed_deaths_check_test[imputed_deaths_check_test["Deaths"] > 10])

# Ensure no NaN values
# Rows with missing values
rows_with_missing = mort_pop_no_missing_test[
    mort_pop_no_missing_test.isna().any(axis=1)
]
print(rows_with_missing)

,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,death_rate
0,01001,3,6,Alabama,Autauga County,2003,46800,0.0,0.000000
1,01003,3,6,Alabama,Baldwin County,2003,151509,0.0,0.000000
2,01005,3,6,Alabama,Barbour County,2003,28594,0.0,0.000000
3,01007,3,6,Alabama,Bibb County,2003,21399,0.0,0.000000
4,01009,3,6,Alabama,Blount County,2003,53457,0.0,0.000000
...,...,...,...,...,...,...,...,...,...
37953,56037,4,8,Wyoming,Sweetwater County,2015,44719,3.0,0.000061
37954,56039,4,8,Wyoming,Teton County,2015,23047,1.0,0.000061
37955,56041,4,8,Wyoming,Uinta County,2015,20763,1.0,0.000061
37956,56043,4,8,Wyoming,Washakie County,2015,8278,1.0,0.000061


,FIPS_CODE,REGION,DIVISION,ST_NAME,CTY_NAME,Year,Population,Deaths,death_rate


Empty DataFrame
Columns: [FIPS_CODE, REGION, DIVISION, ST_NAME, CTY_NAME, Year, Population, Deaths, death_rate]
Index: []
